# Capstone Project - The Battle of Neighborhoods

## Introduction/Business Problem

Singapore is a highly developed city state with no shortage of amenities. It is further divided into 23 towns and 3 estates, each with their own distinct flavor.

At 721.5 square kilometers, one would think that the average Singapore would know what differentiates each town from one another (other than price of course!). But alas, this is not the case. 

As a young Singaporean looking to buy a house, I faced the daunting task of comparing houses from different towns based on price as well as amenities. 

We will use data from various sources to generate group towns with similar amenities into clusters as well as generate a choropleth map of all the towns based on median resale prices of the towns.

From there on, potential home buyers will be able to have a broad overview of the median house values for each town, together with the amenities that define the particular town. This allows buyers to select towns which are cheaper and yet, have the same types of amenities that they want.

## Data

### The following data sets will be used:
1.   Location data from Foursquare
2.   Median resale flat prices
3.   Geojson data for boundaries of each town
4.   Latitude and longitude of each town


### Location data from Foursquare
List of venues for each town will be obtained from Foursquare. The data obtained will be used to determine the venues that define the particular town. This will allow us to employ clustering to group towns with similar venues. 

### Median resale flat prices
We will be using the median resale flat prices from data.gov.sg to allow us to determine what is the average sale price of flats in each particular town. Data obtained from https://data.gov.sg/dataset/median-resale-prices-for-registered-applications-by-town-and-flat-type

### Geojson data for boundaries of each town
Geojson data will be used to define the boundaries of each town when generating the chloropleth map based on the median resale flat prices. Data obtained from https://github.com/yinshanyang/singapore

### Latitude and longitude of each town
Latitude and longitude values of each town allows us to get the list of venues from Foursquare. This data is tabulated manually, using Google to get the coordinates of each individual town. 

## Methodology

### Importing the necessary modules

In [1]:
# Importing modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim
import requests

### Importing data required

We import median resale price and coords for each town.

In [2]:
# importing data
na_vals = ['-', 'na', '0']
resale = pd.read_csv('/content/median-resale-prices-for-registered-applications-by-town-and-flat-type.csv', na_values=na_vals)
coords = pd.read_csv('/content/singapore_town_coords.csv')

print('Data sets imported successfully!')


Data sets imported successfully!


In [3]:
print('Size of dataframe:', resale.shape)

resale.head()



Size of dataframe: (8268, 4)


,quarter,town,flat_type,price
0,2007-Q2,Ang Mo Kio,1-room,NaN
1,2007-Q2,Ang Mo Kio,2-room,NaN
2,2007-Q2,Ang Mo Kio,3-room,172000.0
3,2007-Q2,Ang Mo Kio,4-room,260000.0
4,2007-Q2,Ang Mo Kio,5-room,372000.0


In [4]:
print('Size of dataframe:', coords.shape)
coords.head()

Size of dataframe: (26, 3)


,town,Latitude,Longitude
0,ANG MO KIO,1.3691,103.8454
1,BEDOK,1.3236,103.9273
2,BISHAN,1.3526,103.8352
3,BUKIT BATOK,1.3590,103.7637
4,BUKIT MERAH,1.2819,103.8239


### Cleaning up the resale data

Removing rows with NaN values.

In [5]:
# removing rows containing NaN
resale.dropna(inplace=True)
resale.reset_index(drop=True,inplace=True)
resale.head()

,quarter,town,flat_type,price
0,2007-Q2,Ang Mo Kio,3-room,172000.0
1,2007-Q2,Ang Mo Kio,4-room,260000.0
2,2007-Q2,Ang Mo Kio,5-room,372000.0
3,2007-Q2,Bedok,3-room,172000.0
4,2007-Q2,Bedok,4-room,224500.0


Converting columns 'town and 'flat_type' to uppercase letters for standardisation.

In [6]:
# Convert town names to uppercase
resale['town'] = resale.town.apply(lambda x: x.upper())
print(resale['town'].unique())

# Convert flat_type to uppercase
resale['flat_type'] = resale.flat_type.apply(lambda x: x.upper())
print(resale['flat_type'].unique())

['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT PANJANG'
 'CENTRAL' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG' 'HOUGANG' 'JURONG EAST'
 'JURONG WEST' 'KALLANG/WHAMPOA' 'MARINE PARADE' 'PASIR RIS' 'PUNGGOL'
 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG' 'SERANGOON' 'TAMPINES' 'TOA PAYOH'
 'WOODLANDS' 'YISHUN' 'CENTRAL AREA']
['3-ROOM' '4-ROOM' '5-ROOM' 'EXECUTIVE' '2-ROOM' 'EXEC']


In Singapore, there are executive flats as well as executive mansionette (2 story flats). However, the data provided does not indicate as such. Hence, we shall rename 'exec' to 'executive' for methodical purposes. As we are taking the median resale price based later on, this will not affect the results.  

In [7]:
# Renaming exec to executive
resale['flat_type'].replace({'EXEC':'EXECUTIVE'},inplace=True)
print(resale['flat_type'].unique())

['3-ROOM' '4-ROOM' '5-ROOM' 'EXECUTIVE' '2-ROOM']


Renaming 'kallang/whampoa' to 'kallang' for consistency

In [8]:
# Renaming kallang/whampoa to kallang
resale['town'].replace({'KALLANG/WHAMPOA':'KALLANG'},inplace=True)
print(resale['town'].unique())

['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT PANJANG'
 'CENTRAL' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG' 'HOUGANG' 'JURONG EAST'
 'JURONG WEST' 'KALLANG' 'MARINE PARADE' 'PASIR RIS' 'PUNGGOL'
 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG' 'SERANGOON' 'TAMPINES' 'TOA PAYOH'
 'WOODLANDS' 'YISHUN' 'CENTRAL AREA']


As we are getting the mean of price for each flat type for each town, the date does not matter. Hence we will remove the 'quarter' column.

In [9]:
# Dropping quarter column as it is not required
resale.drop(columns='quarter', inplace=True)


Removing 'central area' and renaming 'central' to 'downtown core' for consistency. 

In [10]:
# Removing central area due to duplicate
filt = resale['town'] == 'CENTRAL AREA'
resale.drop(index=resale[filt].index, inplace=True)

# Renaming central to downtown core
resale['town'].replace({'CENTRAL':'DOWNTOWN CORE'},inplace=True)
print(resale['town'].unique())

['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT PANJANG'
 'DOWNTOWN CORE' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG' 'HOUGANG'
 'JURONG EAST' 'JURONG WEST' 'KALLANG' 'MARINE PARADE' 'PASIR RIS'
 'PUNGGOL' 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG' 'SERANGOON' 'TAMPINES'
 'TOA PAYOH' 'WOODLANDS' 'YISHUN']


Grouping the data based on 'town' and mean resale price.

In [11]:
# Median resale value by town
resale_sorted = resale.groupby(['town'], as_index=False).mean()
resale_sorted

,town,price
0,ANG MO KIO,431908.552632
1,BEDOK,413190.643275
2,BISHAN,545377.876106
3,BUKIT BATOK,400044.642857
4,BUKIT MERAH,547157.232704
5,BUKIT PANJANG,400523.255814
6,CHOA CHU KANG,410405.263158
7,CLEMENTI,414637.719298
8,DOWNTOWN CORE,485609.756098
9,GEYLANG,381505.785124


The resale data is now ready for use for the visualisation later on.

### Cleaning up coords data



Removing rows with NaN values.

In [12]:
# Removing nan values
coords.dropna(inplace=True)

Removing 'central area' as it is a duplicate.

In [13]:
# Removing duplicates
filt = coords['town'] == 'CENTRAL AREA'
coords.drop(index=coords[filt].index, inplace=True)
coords.reset_index(drop=True, inplace=True)

Again, as per the resale data, we will rename 'kallang/whampoa' and 'central' to 'kallang' and 'downtown core' respectively.

In [14]:
# Renaming data 
coords['town'].replace({'KALLANG/WHAMPOA':'KALLANG'},inplace=True)
coords['town'].replace({'CENTRAL':'DOWNTOWN CORE'},inplace=True)

Data is cleaned up and ready to use.

In [15]:
coords

,town,Latitude,Longitude
0,ANG MO KIO,1.3691,103.8454
1,BEDOK,1.3236,103.9273
2,BISHAN,1.3526,103.8352
3,BUKIT BATOK,1.3590,103.7637
4,BUKIT MERAH,1.2819,103.8239
5,BUKIT PANJANG,1.3774,103.7719
6,DOWNTOWN CORE,1.2789,103.8536
7,CHOA CHU KANG,1.3840,103.7470
8,CLEMENTI,1.3162,103.7649
9,GEYLANG,1.3201,103.8918


We have sucessfully cleaned up the data and get it to the state we want it to be. Now, we will use Foursquare to populate the venues for each town.



### Getting the venues using Foursquare

Firstly, we need to gather the location of Singapore. We will be using Geolocator for this.

In [16]:
# Getting coords of Singapore
address = 'Singapore, SG'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Singapore are 1.2904753, 103.8520359.


Getting the login credentials for Foursquare

In [17]:
# Foursquare login
CLIENT_ID = 'GNJE3HSGUFIQMXOLVWXK2SSHCGSCNSR3BB050DX0S3JT25DR' # your Foursquare ID
CLIENT_SECRET = 'HR4HIKY3ET5Q0DH4OH4W5UU4KHFYEA5ZPAD0XTSLSIGBHTYX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GNJE3HSGUFIQMXOLVWXK2SSHCGSCNSR3BB050DX0S3JT25DR
CLIENT_SECRET:HR4HIKY3ET5Q0DH4OH4W5UU4KHFYEA5ZPAD0XTSLSIGBHTYX


Generating the search URL for a town. We will be using Bukit Timah as an example for this. The limit and radius will be set to 100 and 500 respectively.

In [18]:
# URL for search using Bukit Timah
LIMIT = 100
radius = 500
#near = 'Bukit Timah'
neighbourhood_latitude = 1.3408630000000001
neighbourhood_longitude = 103.83039182212079
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude,
    neighbourhood_longitude,
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=GNJE3HSGUFIQMXOLVWXK2SSHCGSCNSR3BB050DX0S3JT25DR&client_secret=HR4HIKY3ET5Q0DH4OH4W5UU4KHFYEA5ZPAD0XTSLSIGBHTYX&v=20180605&ll=1.3408630000000001,103.83039182212079&radius=500&limit=100'

Getting the results in a JSON file.

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4b572c0e78567a46cadb5c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4f6d1d26e4b0ca47b0ab88a4-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/lake_',
          'suffix': '.png'},
         'id': '56aa371be4b08b9a8d573541',
         'name': 'Reservoir',
         'pluralName': 'Reservoirs',
         'primary': True,
         'shortName': 'Reservoir'}],
       'id': '4f6d1d26e4b0ca47b0ab88a4',
       'location': {'address': 'MacRitchie Reservoir Park',
        'cc': 'SG',
        'city': 'Singapore',
        'country': 'Singapore',
        'distance': 414,
        'formattedAddress': ['MacRitchie Reservoir Park', 'Singapore'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 1.3426982080671415,
      

We will now extract the category of each venue

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Generating a dataframe that contains the name, category and coords of each venue in the town of Bukit Batok.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,MacRitchie Reservoir,Reservoir,1.342698,103.833632
1,MacRitchie Reservoir Park,Park,1.341727,103.834281
2,Mediacorp dance studio,Dance Studio,1.340638,103.831733
3,Bus Stop 51119 (Bef Andrew Rd),Bus Station,1.340174,103.832537
4,MediaCorp Production Resource,Theater,1.337807,103.832817


Now, we shall repeat the above steps for all towns in Singapore.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

singapore_venues = getNearbyVenues(names=coords['town'],
                                   latitudes=coords['Latitude'],
                                   longitudes=coords['Longitude']
                                  )

ANG MO KIO
BEDOK
BISHAN
BUKIT BATOK
BUKIT MERAH
BUKIT PANJANG
DOWNTOWN CORE
CHOA CHU KANG
CLEMENTI
GEYLANG
HOUGANG
JURONG EAST
JURONG WEST
KALLANG
MARINE PARADE
PASIR RIS
PUNGGOL
QUEENSTOWN
SEMBAWANG
SENGKANG
SERANGOON
TAMPINES
TOA PAYOH
WOODLANDS
YISHUN


Checking the size of dataframe.

In [23]:
singapore_venues.shape
singapore_venues

,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ANG MO KIO,1.3691,103.8454,Kam Jia Zhuang Restaurant,1.368167,103.844118,Asian Restaurant
1,ANG MO KIO,1.3691,103.8454,Old Chang Kee,1.369094,103.848389,Snack Place
2,ANG MO KIO,1.3691,103.8454,FairPrice Xtra,1.369279,103.848886,Supermarket
3,ANG MO KIO,1.3691,103.8454,MOS Burger,1.369170,103.847831,Burger Joint
4,ANG MO KIO,1.3691,103.8454,NTUC FairPrice,1.371507,103.847082,Supermarket
...,...,...,...,...,...,...,...
1298,YISHUN,1.4304,103.8354,Sheng Siong Supermarket,1.426878,103.837051,Grocery Store
1299,YISHUN,1.4304,103.8354,Choh Dee Place,1.435287,103.838208,Coffee Shop
1300,YISHUN,1.4304,103.8354,Blk 110 Yishun Coffeeshop 优来咖啡店,1.433219,103.829558,Coffee Shop
1301,YISHUN,1.4304,103.8354,Bus Stop 59311 (Blk 760),1.426232,103.832152,Bus Stop


Generating the unique counts for each town.

In [24]:
singapore_venues.groupby('Town').count()

,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Town,,,,,,
ANG MO KIO,61,61,61,61,61,61
BEDOK,79,79,79,79,79,79
BISHAN,74,74,74,74,74,74
BUKIT BATOK,36,36,36,36,36,36
BUKIT MERAH,46,46,46,46,46,46
BUKIT PANJANG,16,16,16,16,16,16
CHOA CHU KANG,31,31,31,31,31,31
CLEMENTI,66,66,66,66,66,66
DOWNTOWN CORE,100,100,100,100,100,100


We will now analyse each town by using one hot encoding. 

In [25]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Town'] = singapore_venues['Town'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()



,Town,Accessories Store,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Campground,Canal,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Church,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Dumpling Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Music Store,Music Venue,Night Market,Nightclub,Noodle House,Office,Outdoor Supply Store,Outdoors & Recreation,Outlet Store,Park,Peking Duck Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Tea Room,Temple,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wings Joint,Yoga Studio
0,ANG MO KIO,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Converting each venue into frequency.

In [26]:
singapore_grouped = singapore_onehot.groupby('Town').mean().reset_index()
singapore_grouped

,Town,Accessories Store,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Campground,Canal,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Church,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Dumpling Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Music Store,Music Venue,Night Market,Nightclub,Noodle House,Office,Outdoor Supply Store,Outdoors & Recreation,Outlet Store,Park,Peking Duck Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Tea Room,Temple,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wings Joint,Yoga Studio
0,ANG MO KIO,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.032787,0.000000,0.000000,0.000000,0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.016393,0.00,0.032787,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.016393,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.065574,0.00,0.00,0.000000,0.000000,0.00,0.081967,0.000000,0.000000,0.00,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.049180,0.000000,0.000000,0.016393,0.000000,0.016393,0.00,0.032787,0.000000,0.000000,0.000000,0.000000,0.000000,0.098361,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.016393,0.000000,0.016393,0.00,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.01639

Getting the top 5 venues for each town, based on frequency.

In [27]:
num_top_venues = 5

for town in singapore_grouped['Town']:
    print("----"+town+"----")
    temp = singapore_grouped[singapore_grouped['Town'] == town].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----ANG MO KIO----
                venue  freq
0          Food Court  0.10
1         Coffee Shop  0.08
2  Chinese Restaurant  0.07
3        Dessert Shop  0.05
4        Noodle House  0.03


----BEDOK----
                venue  freq
0         Coffee Shop  0.08
1          Food Court  0.06
2  Chinese Restaurant  0.05
3         Supermarket  0.05
4              Bakery  0.04


----BISHAN----
                venue  freq
0  Chinese Restaurant  0.14
1                Café  0.09
2         Coffee Shop  0.05
3     Thai Restaurant  0.05
4        Dessert Shop  0.04


----BUKIT BATOK----
                    venue  freq
0                    Café  0.11
1      Italian Restaurant  0.08
2       Indian Restaurant  0.06
3           Shopping Mall  0.06
4  Thrift / Vintage Store  0.06


----BUKIT MERAH----
                 venue  freq
0   Chinese Restaurant  0.13
1          Coffee Shop  0.09
2     Sushi Restaurant  0.04
3  Japanese Restaurant  0.04
4                 Café  0.04


----BUKIT PANJANG----
          

Converting list to dataframe.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Getting the top 10 venues for each town in a dataframe.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
town_venues_sorted = pd.DataFrame(columns=columns)
town_venues_sorted['Town'] = singapore_grouped['Town']

for ind in np.arange(singapore_grouped.shape[0]):
    town_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

town_venues_sorted

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANG MO KIO,Food Court,Coffee Shop,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Snack Place,Japanese Restaurant,Supermarket,Noodle House,Vegetarian / Vegan Restaurant
1,BEDOK,Coffee Shop,Food Court,Supermarket,Chinese Restaurant,Fast Food Restaurant,Café,Asian Restaurant,Sandwich Place,Bakery,Seafood Restaurant
2,BISHAN,Chinese Restaurant,Café,Thai Restaurant,Coffee Shop,Bakery,Food Court,Indian Restaurant,Ice Cream Shop,Dessert Shop,Supermarket
3,BUKIT BATOK,Café,Italian Restaurant,Coffee Shop,Thrift / Vintage Store,Indian Restaurant,Shopping Mall,Supermarket,American Restaurant,Bus Line,Fast Food Restaurant
4,BUKIT MERAH,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Café,Bookstore,Bakery
5,BUKIT PANJANG,Park,Food Court,Miscellaneous Shop,Residential Building (Apartment / Condo),Market,Gym,Bike Trail,Bus Station,Coffee Shop,College Cafeteria
6,CHOA CHU KANG,Fast Food Restaurant,Coffee Shop,Bus Station,Portuguese Restaurant,Japanese Restaurant,Bus Line,Snack Place,Sandwich Place,Bubble Tea Shop,Light Rail Station
7,CLEMENTI,Food Court,Chinese Restaurant,Asian Restaurant,Dessert Shop,Dim Sum Restaurant,Fast Food Restaurant,Chinese Breakfast Place,Gym,Japanese Restaurant,Coffee Shop
8,DOWNTOWN CORE,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym / Fitness Center,Waterfront,Chinese Restaurant,Gym,Southern / Soul Food Restaurant,Food Court
9,GEYLANG,Chinese Restaurant,Fast Food Restaurant,Asian Restaurant,Food Court,Shopping Mall,Noodle House,Supermarket,Coffee Shop,Dessert Shop,Bubble Tea Shop


We now have a dataframe that contains the top 10 venues of each town. We can now use sklearn to cluster the data.

### Clustering the towns into 5 clusters

Kmeans clustering is used to group towns with similar venues into separate clusters. This allows us to compare towns with similar venues and their respective price.

In [30]:
# set number of clusters
kclusters = 5

singapore_grouped_clustering = singapore_grouped.drop('Town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 0, 1, 3, 0, 0, 0, 1], dtype=int32)

Merging the cluster labels with dataframe containing top 10 venues.

In [31]:
town_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

singapore_merged = coords

# merge singapore_grouped with coords to add latitude/longitude for each town
singapore_merged = singapore_merged.join(town_venues_sorted.set_index('Town'), on='town')


singapore_merged.head() # check the last columns!

,town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANG MO KIO,1.3691,103.8454,1,Food Court,Coffee Shop,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Snack Place,Japanese Restaurant,Supermarket,Noodle House,Vegetarian / Vegan Restaurant
1,BEDOK,1.3236,103.9273,0,Coffee Shop,Food Court,Supermarket,Chinese Restaurant,Fast Food Restaurant,Café,Asian Restaurant,Sandwich Place,Bakery,Seafood Restaurant
2,BISHAN,1.3526,103.8352,1,Chinese Restaurant,Café,Thai Restaurant,Coffee Shop,Bakery,Food Court,Indian Restaurant,Ice Cream Shop,Dessert Shop,Supermarket
3,BUKIT BATOK,1.3590,103.7637,0,Café,Italian Restaurant,Coffee Shop,Thrift / Vintage Store,Indian Restaurant,Shopping Mall,Supermarket,American Restaurant,Bus Line,Fast Food Restaurant
4,BUKIT MERAH,1.2819,103.8239,1,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Café,Bookstore,Bakery


Visualising the clusters on a map.

In [32]:
sg_geo = r"/content/singapore_planning_area.geojson"


sg_map = folium.Map(location=[1.3408630000000001, 103.83039182212079], zoom_start=11, tiles='cartodbpositron')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['town'], singapore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(sg_map)

choropleth = folium.Choropleth(
    geo_data=sg_geo,
    name='Median price',
    data=resale_sorted,
    columns=['town', 'price'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name='Median Price of flats for each town ($)'
).add_to(sg_map)

#Adding tooltips for each town
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name',],labels=False)
)

folium.LayerControl().add_to(sg_map)
sg_map

The map contains two layers, the cluster layer and choropleth layer.

## Results

Five clusters were generated. The towns for each cluster can be viewed below. 

### Cluster 0

In [33]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 0, singapore_merged.columns[[0] + list(range(3, singapore_merged.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,BEDOK,0,Coffee Shop,Food Court,Supermarket,Chinese Restaurant,Fast Food Restaurant,Café,Asian Restaurant,Sandwich Place,Bakery,Seafood Restaurant
3,BUKIT BATOK,0,Café,Italian Restaurant,Coffee Shop,Thrift / Vintage Store,Indian Restaurant,Shopping Mall,Supermarket,American Restaurant,Bus Line,Fast Food Restaurant
6,DOWNTOWN CORE,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym / Fitness Center,Waterfront,Chinese Restaurant,Gym,Southern / Soul Food Restaurant,Food Court
7,CHOA CHU KANG,0,Fast Food Restaurant,Coffee Shop,Bus Station,Portuguese Restaurant,Japanese Restaurant,Bus Line,Snack Place,Sandwich Place,Bubble Tea Shop,Light Rail Station
8,CLEMENTI,0,Food Court,Chinese Restaurant,Asian Restaurant,Dessert Shop,Dim Sum Restaurant,Fast Food Restaurant,Chinese Breakfast Place,Gym,Japanese Restaurant,Coffee Shop
11,JURONG EAST,0,Japanese Restaurant,Coffee Shop,Chinese Restaurant,Shopping Mall,Café,Food Court,Clothing Store,Sushi Restaurant,Fast Food Restaurant,Korean Restaurant
12,JURONG WEST,0,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Chinese Restaurant,Dessert Shop,Coffee Shop,Indian Restaurant,Café,Food Court,Spa
14,MARINE PARADE,0,Chinese Restaurant,Italian Restaurant,Pizza Place,Café,Noodle House,Indian Restaurant,Supermarket,Bakery,Ice Cream Shop,Bar
15,PASIR RIS,0,Food Court,Bus Station,Fast Food Restaurant,Coffee Shop,Pharmacy,Sandwich Place,Italian Restaurant,Park,Supermarket,Recreation Center
20,SERANGOON,0,Café,Coffee Shop,Asian Restaurant,Ice Cream Shop,Japanese Restaurant,Clothing Store,Park,Multiplex,Indonesian Restaurant,Pharmacy


### Cluster 1

In [34]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 1, singapore_merged.columns[[0] + list(range(3, singapore_merged.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANG MO KIO,1,Food Court,Coffee Shop,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Snack Place,Japanese Restaurant,Supermarket,Noodle House,Vegetarian / Vegan Restaurant
2,BISHAN,1,Chinese Restaurant,Café,Thai Restaurant,Coffee Shop,Bakery,Food Court,Indian Restaurant,Ice Cream Shop,Dessert Shop,Supermarket
4,BUKIT MERAH,1,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Seafood Restaurant,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Café,Bookstore,Bakery
9,GEYLANG,1,Chinese Restaurant,Fast Food Restaurant,Asian Restaurant,Food Court,Shopping Mall,Noodle House,Supermarket,Coffee Shop,Dessert Shop,Bubble Tea Shop
10,HOUGANG,1,Coffee Shop,Chinese Restaurant,Noodle House,Food Court,Café,Asian Restaurant,Indian Restaurant,Dessert Shop,Snack Place,Bakery
13,KALLANG,1,Coffee Shop,BBQ Joint,Noodle House,Hostel,Asian Restaurant,Restaurant,Supermarket,Vietnamese Restaurant,Fast Food Restaurant,Food Court
17,QUEENSTOWN,1,Coffee Shop,Sandwich Place,Café,Food Court,Korean Restaurant,Vietnamese Restaurant,Beer Bar,Asian Restaurant,Noodle House,Frozen Yogurt Shop
18,SEMBAWANG,1,Coffee Shop,Bus Station,Fast Food Restaurant,Pizza Place,Park,Supermarket,Chinese Restaurant,Café,Government Building,Shopping Mall
19,SENGKANG,1,Food Court,Coffee Shop,Metro Station,Wings Joint,Fast Food Restaurant,Restaurant,Bus Line,Convenience Store,Shopping Mall,Café
22,TOA PAYOH,1,Seafood Restaurant,Chinese Restaurant,Noodle House,Coffee Shop,Food Court,Bus Stop,Asian Restaurant,Market,Dessert Shop,Pizza Place


### Cluster 2

In [35]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 2, singapore_merged.columns[[0] + list(range(3, singapore_merged.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,PUNGGOL,2,Fast Food Restaurant,Japanese Restaurant,Playground,Steakhouse,High School,Sandwich Place,Chinese Restaurant,Bus Stop,Supermarket,Bagel Shop


### Cluster 3

In [36]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 3, singapore_merged.columns[[0] + list(range(3, singapore_merged.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,BUKIT PANJANG,3,Park,Food Court,Miscellaneous Shop,Residential Building (Apartment / Condo),Market,Gym,Bike Trail,Bus Station,Coffee Shop,College Cafeteria


### Cluster 4

In [37]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 4, singapore_merged.columns[[0] + list(range(3, singapore_merged.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,TAMPINES,4,Coffee Shop,Fast Food Restaurant,Shopping Mall,Bakery,Housing Development,Basketball Court,Pool,Deli / Bodega,Chinese Restaurant,Sandwich Place


## Discussion

### Observations made:


1.   Price of resale flats
2.   Clusters



### 1.     Price of resale flats
From the visualisation, it is evident that flats near the central area and south are the most expensive. These are the so called 'mature estates'. These estates have been around for some time and are expensive due to flat configuration, flat size and proximity to town (Orchard and the Central Business District).

Towns in the west such as Jurong, are generally cheaper compared to the rest due to close proximity to Tuas industrial areas. However, with the development of the shopping hub in Jurong East as well as a new town Tengah, the prices in the area may see an increase.

Towns in the east consists of a mix of median resale prices with towns such as Sengkang, Punggol, Pasir Ris and Tampines fetching a higher price.

Towns in the north such as Yishun and Woodlands are also considered cheap compared to other sectors due to the accessibility to central and other parts of Singapore. However, they are nearest to the Causeway, making trips to Malaysia easy and convenient for people who reside there.

Black sectors consists of areas where no data was collected in terms of sales of flats. Private housing sales are not taken into account hence areas such as Orchard, Newton .etc are displayed in black. 

### 2.     Clusters
5 clusters were selected for the clustering. For cluster 1 and 2, we can see a good mix of old and new towns less cluster 2, 3 and 4. 

There are several reasons for this. First, we are using Foursquare location data. This data itself is contributed by the public. As such, the number of venues are determined by how active users are in reporting and rating the venues.

Secondly, there is a lack of distinct types of venues in Singapore as it contains little to no nature features (less parks in towns). Most of the nature features such are located in areas with no sale of flats. As a city state, there are no shortage of cafes, restaurants and shopping centres hence lack of diversity (most towns will have cafes, food centres as their top 10 venues). 

But nevertheless, we are able to use data from cluster 0 and 1 to find towns with similiar venues but at a lower cost.

## Conclusion

This visualisation allows potential home buyers to see what is the median resale price of flats in a particular town as well as the common amenities. This will allow buyers to easily select the town they like and then look in-depth for the location they want.

However, the data does not account for private housing sale prices.

Further improvements can be made by generating multiple  layers of choropleth maps based on flat type (3-room, 4-room .etc) and to take into account private housing prices to make it even more informative. Optimisation of number of clusters can be done to futher improve the clustering ability.